In [184]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [185]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes=2):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# 設定參數

In [186]:
input_dim = 3
hidden_dim = 16
num_layers = 2
num_classes = 2
MAX_SEQ_LEN = 10

In [187]:
model = LSTMClassifier(input_dim, hidden_dim, num_layers, num_classes)

In [188]:
model_path = "result/fold_0_model.pth"  # 根據您要使用的模型權重檔案修改此路徑
model.load_state_dict(torch.load(model_path))
model.eval()  # 設定為評估模式

/tmp/ipykernel_8476/2080666277.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


LSTMClassifier(
  (lstm): LSTM(3, 16, num_layers=2, batch_first=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)

In [189]:
test_normal_data_path = "dataset/normal/70.csv"  
test_abnormal_data_path = "dataset/abnormal/XX_線材生鏽.csv" 
custom_data = "test.csv"
df = pd.read_csv(custom_data)

In [190]:
current = df['current'].values
voltage = df['voltage'].values
power = df['power'].values
sequence = np.column_stack((current, voltage, power))

In [191]:
sequences = []
seq_len = sequence.shape[0]
num_chunks = seq_len // MAX_SEQ_LEN
for i in range(num_chunks):
    start = i * MAX_SEQ_LEN
    end = start + MAX_SEQ_LEN
    chunk = sequence[start:end]
    sequences.append(chunk)

new_data = np.array(sequences, dtype=np.float32)

In [192]:
data_tensor = torch.tensor(new_data, dtype=torch.float32)

In [193]:
with torch.no_grad():
    outputs = model(data_tensor)
    _, predicted = torch.max(outputs, 1)

print("Predictions:", predicted)

for i, pred in enumerate(predicted):
    if pred == 1:
        print(f"Sequence {i} is predicted as abnormal.")
    else:
        print(f"Sequence {i} is predicted as normal.")

Predictions: tensor([0, 0, 0, 1])
Sequence 0 is predicted as normal.
Sequence 1 is predicted as normal.
Sequence 2 is predicted as normal.
Sequence 3 is predicted as abnormal.
